In [ ]:
# pip install ipywidgets
# pip install plotly
# pip install ipympl

In [ ]:
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual, GridBox, Layout, VBox
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots

pd.set_option('display.max_columns', None)

%matplotlib inline
%matplotlib widget

# Dataset

In [ ]:
LAND_USE_COLS = ['c3ann', 'c3nfx', 'c3per', 'c4ann', 'pastr', 'range', 'secdf', 'secdn', 'urban']
DIFF_LAND_USE_COLS = [f"{col}_diff" for col in LAND_USE_COLS]
PRESCRIBED_LAND_USE_COLS = [f"{col}_prescribed" for col in LAND_USE_COLS]
OTHER_FEATURES_COLS = ['primf', 'primn', 'cell_area']
ALL_LAND_USE_COLS = ['primf', 'primn'] + LAND_USE_COLS
COLS_MAP = dict(zip(LAND_USE_COLS, DIFF_LAND_USE_COLS))

In [ ]:
DATASET_CSV = '../data/gcb/processed/uk_eluc.csv'
with open(DATASET_CSV) as df_file:
    data_source_df = pd.read_csv(df_file)

In [ ]:
data_source_df.tail()

# Code

In [ ]:
min_lat = data_source_df["i_lat"].min()
max_lat = data_source_df["i_lat"].max()
min_lon = data_source_df["i_lon"].min()
max_lon = data_source_df["i_lon"].max()
min_time = data_source_df["time"].min()
max_time = data_source_df["time"].max()

In [ ]:
# Create the latitude input field
latitude_input = widgets.FloatText(description='Latitude:', value=51.625)

# Create the longitude input field
longitude_input = widgets.FloatText(description='Longitude:', value=-3.375)

# Create the time input field
time_input = widgets.IntText(description='Year:', value=2021)

# Display the widgets
# display(latitude_input, longitude_input, time_input)

# Context
Enter the coordinates of a cell and a year


In [ ]:
print(f"Latitude must be between {min_lat} and {max_lat}, in 0.250 increments.")
print(f"Longitude must be between {min_lon} and {max_lon}, in 0.250 increments.")
print(f"Year must be between {min_time} and {max_time}.")

In [ ]:
display(latitude_input, longitude_input, time_input)

In [ ]:
sample_df = data_source_df[(data_source_df.i_lat==latitude_input.value) & 
                           (data_source_df.i_lon==longitude_input.value) &
                           (data_source_df.time==time_input.value)]
sample_df

In [ ]:
sample_df.iloc[0][LAND_USE_COLS].plot.pie(labels=LAND_USE_COLS, autopct='%1.2f%%')
plt.show()

In [ ]:
land_use = sample_df.iloc[0][LAND_USE_COLS].sum() * 100
print(f"{land_use:.2f}% of the land is used")

### Functions that call a model

In [ ]:
def run_prescriptor(context):
    return dict(zip(LAND_USE_COLS, context))

In [ ]:
def run_predictor(context, actions):
    sum = 0
    for c in context:
        sum += c
    for a in actions:
        sum += a

    return sum

### Demo

In [ ]:
"""
Submits context and creates pie chart
Updates sliders for pie chart accordingly
#TODO: handle case where percentages don't sum to 1
"""
def prescribe(b):
    data = [box.value for box in boxes.values()]
    #print(f"Sum: {sum(data)}")
    # if sum(data) == 100:
    prescribed = run_prescriptor(data)

    for feature in LAND_USE_COLS:
        sliders[feature].value = prescribed[feature]

    # Clear figure and re-plot
    fig.add_trace(go.Pie(values=list(prescribed.values()), 
                     labels=LAND_USE_COLS, 
                     domain=dict(x=[0.5, 1]), 
                     title="Prescribed"), row=1, col=2)

#     else:
#         with(prescribe_output):
#             print("Please make sure your percentage sums to 100")


"""
Locks a slider so it isn't affected by the sum to 100 computation
"""
def lock(change):
    if change["new"]:
        locked.add(change["owner"])
    else:
        locked.remove(change["owner"])

"""
Real-time updater for context pie chat
"""
def update_context_plot(change):
    with fig.batch_update():
        if len(fig["data"]) > 0:
            fig["data"][0]["values"] = [box.value for box in boxes.values()]


"""
Real-time updater for prescribed pie chart
# TODO: this breaks when a value hits 0
"""
def update_presc_plot(change):
    with fig.batch_update():
        if len(fig["data"]) > 1:

            # owner = change["owner"]

            # First compute what percentage is locked
            # locked_sum = 0
            # for feat in sliders:
            #     if sliders[feat] != owner and ticks[feat] in locked:
            #         locked_sum += sliders[feat].value

            # Add locked percentage to old and new because we don't factor
            # them in to the 100% in our calculating the new value
            # old = change["old"] + locked_sum
            # new = change["new"] + locked_sum

            # TODO: Adjust each slider that isn't being currently modified
            # for feat in sliders:
            #     slider = sliders[feat]
            #     tick = ticks[feat]
            #     if slider != owner and tick not in locked:
            #         # Unobserve so we don't infinitely recurse
            #         slider.unobserve(update, names="value")
            #         # old value / old total = new value / new total
            #         slider.value = slider.value / (100 - old) * (100 - new)
            #         slider.observe(update, names="value")

            # Update pie chart
            fig["data"][1]["values"] = [slider.value for slider in sliders.values()]


"""
Submits context and actions and outputs prediction
"""
def predict(b):
    with predict_output:
        context = [box.value for box in boxes.values()]
        actions = [slider.value for slider in sliders.values()]
        outcome = run_predictor(context, actions)
        print("ELUC: ", outcome)
        

"""
Construct widgets and attach them to their functions
TODO: handle unusable land
"""
boxes = {feature : widgets.FloatText(value=round(sample_df[feature].iloc[0] * 100, 2), description=feature) for feature in LAND_USE_COLS}
sliders = {feature : widgets.FloatSlider(value=0.0, description="Prescribed " + feature, style=dict(description_width='initial')) for feature in LAND_USE_COLS}
ticks = {feature : widgets.Checkbox(value=False, description="Lock " + feature, style=dict(description_width='initial')) for feature in LAND_USE_COLS}

# For use in locking and unlocking sliders
locked = set()

prescribe_button = widgets.Button(description="Prescribe")
prescribe_button.on_click(prescribe)

predict_button = widgets.Button(description="Predict")
predict_output = widgets.Output()
predict_button.on_click(predict)


"""
Display Interactables and Figures
TODO: add titles, make layout prettier
"""
fig = go.FigureWidget(make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]]))

for box in boxes.values():
    box.observe(update_context_plot, names="value")
    display(box)

# Plot initial context pie chart
init_data = [box.value for box in boxes.values()]
fig.add_trace(go.Pie(values=init_data, 
                     labels=LAND_USE_COLS, 
                     domain=dict(x=[0, 0.5]), 
                     title="Initial"), row=1, col=1)

display(prescribe_button)

display(fig)

for feat in sliders:
    sliders[feat].observe(update_presc_plot, names="value")
    ticks[feat].observe(lock, names="value")
    display(sliders[feat], ticks[feat])

display(predict_button, predict_output)